# Topic Modling using NPB

## Import needed datasets

In [1]:
import os
import pandas as pd
from gensim import models

/home/amirhossein/anaconda3/envs/NPB/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Define needed functions

In [19]:
def sparse_bbow(words):
    return [(index, value) for index,value in words.items() if value == 1]

## Load Dataset

### cora.cites

In [3]:
data_dir = os.path.expanduser("../Datasets/cora")

edgelist = pd.read_csv(os.path.join(data_dir, "cora.cites"), sep='\t', header=None, names=["target", "source"])
edgelist.head(5)

,target,source
0,35,1033
1,35,103482
2,35,103515
3,35,1050679
4,35,1103960


### cora.content

In [4]:
feature_names = [i for i in range(1433)]
column_names =  feature_names + ["subject"]
node_data = pd.read_csv(os.path.join(data_dir, "cora.content"), sep='\t', header=None, names=column_names)
node_data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,subject
31336,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1061127,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
1106406,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
13195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
37879,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods


## Transform dataset to ideal format

In [20]:
bbow_corpus = [sparse_bbow(row[1]) for row in node_data.iterrows()]
print(bbow_corpus)

[[(118, 1), (125, 1), (176, 1), (252, 1), (351, 1), (456, 1), (507, 1), (521, 1), (619, 1), (648, 1), (698, 1), (702, 1), (734, 1), (845, 1), (902, 1), (1205, 1), (1209, 1), (1236, 1), (1352, 1), (1426, 1)], [(12, 1), (509, 1), (620, 1), (763, 1), (882, 1), (893, 1), (978, 1), (1131, 1), (1135, 1), (1177, 1), (1207, 1), (1256, 1), (1263, 1), (1266, 1), (1332, 1), (1389, 1), (1425, 1)], [(45, 1), (209, 1), (212, 1), (239, 1), (292, 1), (394, 1), (510, 1), (514, 1), (581, 1), (621, 1), (623, 1), (638, 1), (1075, 1), (1132, 1), (1177, 1), (1206, 1), (1263, 1), (1289, 1), (1349, 1), (1389, 1), (1415, 1), (1421, 1)], [(41, 1), (93, 1), (99, 1), (149, 1), (594, 1), (617, 1), (624, 1), (648, 1), (874, 1), (915, 1), (942, 1), (988, 1), (1004, 1), (1049, 1), (1071, 1), (1170, 1), (1177, 1), (1194, 1), (1292, 1), (1348, 1), (1349, 1)], [(44, 1), (122, 1), (135, 1), (153, 1), (364, 1), (396, 1), (402, 1), (474, 1), (507, 1), (619, 1), (661, 1), (699, 1), (701, 1), (828, 1), (1066, 1), (1174, 1), 

In [22]:
tfidf = models.TfidfModel(bbow_corpus)
print(tfidf[sparse_bbow(node_data.iloc[0])])

[(118, 0.23814769376007022), (125, 0.20535402833474178), (176, 0.2220479218781867), (252, 0.2819939177104131), (351, 0.27571578988483514), (456, 0.23923943345356208), (507, 0.08560014618353899), (521, 0.2106632550168035), (619, 0.14671066466242966), (648, 0.1912097976554182), (698, 0.18496429700987266), (702, 0.2072918321734374), (734, 0.2569841476824823), (845, 0.3455025756073674), (902, 0.31651196101521617), (1205, 0.2072918321734374), (1209, 0.09462368861676156), (1236, 0.23814769376007022), (1352, 0.14018728131527128), (1426, 0.1953932095032192)]
